In [20]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import pandas as pd

# importa desde tu paquete (el nombre debe coincidir con tu __init__.py)
from Isea import ScatterBrush   # si tu paquete se llama distinto, ajusta aquí

# lee tu CSV local
df = pd.read_csv("Isea/datos_vis.csv")

# lanza el scatter con brush (área)
ScatterBrush(df, x="horsepower", y="weight", color="origin")


<IPython.core.display.Javascript object>

In [22]:
from IPython.display import HTML, Javascript, display
display(HTML("<div id='ok' style='padding:6px;border:1px solid #ddd'>Esperando JS…</div>"))
display(Javascript("document.getElementById('ok').innerText = 'JS OK ✅';"))


<IPython.core.display.Javascript object>

In [13]:
from IPython.display import Javascript, display
display(Javascript("""
(async () => {
  const mod = await import('https://cdn.jsdelivr.net/npm/d3@7/+esm');
  const d3v = mod.default ?? mod;
  document.getElementById('ok').innerText = 'D3 cargado ✅ (keys: ' + Object.keys(d3v).length + ')';
})();
"""))


<IPython.core.display.Javascript object>

In [23]:
import pandas as pd
from Isea import RadarChart

radar_data = pd.DataFrame([
    {"name": "Jugador A", "velocidad": 80, "fuerza": 90, "resistencia": 70, "técnica": 85, "precisión": 75},
    {"name": "Jugador B", "velocidad": 95, "fuerza": 60, "resistencia": 85, "técnica": 70, "precisión": 90}
])

RadarChart(
    df=radar_data,
    features=["velocidad", "fuerza", "resistencia", "técnica", "precisión"],
    width=700,
    height=700
)

<IPython.core.display.Javascript object>

In [25]:
import pandas as pd
from Isea.graphs import ScatterBrush, RadarChart, Sunburst, StreamGraph

# Cargar datos
df = pd.read_csv("Isea/datos_vis.csv")
print(f"Datos cargados: {len(df)} filas")
df.head()

Datos cargados: 398 filas


,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


In [26]:
df_sunburst = df.copy()

# Crear categoría de peso
df_sunburst['weight_category'] = pd.cut(
    df_sunburst['weight'], 
    bins=3, 
    labels=['Ligero', 'Medio', 'Pesado']
)

Sunburst(
    df=df_sunburst,
    path_columns=['origin', 'cylinders', 'weight_category'],
    value_column='horsepower',
    width=800,
    height=800,
    color_scheme='schemeSet3'
)

<IPython.core.display.Javascript object>

In [27]:
df_stream = df.groupby(['year', 'origin'])['horsepower'].mean().reset_index()

StreamGraph(
    df=df_stream,
    x_column='year',
    y_column='horsepower',
    category_column='origin',
    width=900,
    height=500,
    color_scheme='schemeTableau10'
)

KeyError: 'year'

In [28]:
df_sunburst2 = df[['cylinders', 'origin', 'mpg']].copy()
df_sunburst2['cylinders'] = df_sunburst2['cylinders'].astype(str) + ' cyl'

Sunburst(
    df=df_sunburst2,
    path_columns=['cylinders', 'origin'],
    value_column='mpg',
    width=700,
    height=700,
    color_scheme='schemePastel1'
)

<IPython.core.display.Javascript object>

In [31]:
df_stream2 = df.copy()
df_stream2['cylinders_cat'] = df_stream2['cylinders'].astype(str) + ' cyl'
df_stream2 = df_stream2.groupby(['model_year', 'cylinders_cat'])['acceleration'].mean().reset_index()

StreamGraph(
    df=df_stream2,
    x_column='year',
    y_column='acceleration',
    category_column='cylinders_cat',
    width=950,
    height=550,
    color_scheme='schemeCategory10'
)

KeyError: ['year']

In [30]:
radar_data = df.groupby('origin')[['horsepower', 'weight', 'acceleration', 'mpg']].mean().reset_index()
radar_data.columns = ['name', 'caballos', 'peso', 'aceleracion', 'eficiencia']

RadarChart(
    df=radar_data,
    features=['caballos', 'peso', 'aceleracion', 'eficiencia'],
    name_col='name',
    width=700,
    height=700
)

<IPython.core.display.Javascript object>

In [32]:
# Celda 1: Cargar datos y aplicar PCA para mejores visualizaciones
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from Isea.graphs import ScatterBrush, RadarChart, Sunburst, StreamGraph

# Cargar datos
df = pd.read_csv("Isea/datos_vis.csv")

# Limpiar datos (eliminar filas con valores faltantes en horsepower)
df = df.dropna(subset=['horsepower'])

print(f"Datos cargados: {len(df)} filas")
print(f"Columnas: {df.columns.tolist()}")

# Aplicar PCA a las características numéricas
numeric_cols = ['mpg', 'displacement', 'horsepower', 'weight', 'acceleration']
X = df[numeric_cols].dropna()

# Estandarizar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA a 2 componentes
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Crear DataFrame con componentes PCA
df_pca = df.loc[X.index].copy()
df_pca['PC1'] = X_pca[:, 0]
df_pca['PC2'] = X_pca[:, 1]

print(f"\nVarianza explicada por PCA:")
print(f"PC1: {pca.explained_variance_ratio_[0]:.2%}")
print(f"PC2: {pca.explained_variance_ratio_[1]:.2%}")
print(f"Total: {sum(pca.explained_variance_ratio_):.2%}")

df_pca.head()

Datos cargados: 392 filas
Columnas: ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin', 'name']

Varianza explicada por PCA:
PC1: 78.54%
PC2: 14.24%
Total: 92.78%


,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,PC1,PC2
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,1.875970,-0.648558
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,2.852657,-0.675575
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,2.262764,-1.043147
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst,2.188684,-0.666426
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino,2.187813,-1.146458


In [33]:
# Celda 2: SCATTER con PCA - Visualización de componentes principales
ScatterBrush(
    df=df_pca, 
    x="PC1", 
    y="PC2", 
    color="origin",
    width=900,
    height=600
)

<IPython.core.display.Javascript object>

In [34]:
# Celda 3: SUNBURST - Jerarquía: origen → cilindros → rango de peso
df_sunburst = df.copy()

# Crear categorías de peso
df_sunburst['weight_range'] = pd.cut(
    df_sunburst['weight'], 
    bins=[0, 2500, 3500, 5500], 
    labels=['Ligero (<2500)', 'Medio (2500-3500)', 'Pesado (>3500)']
)

# Crear categorías de cilindros
df_sunburst['cyl_category'] = df_sunburst['cylinders'].astype(str) + ' cilindros'

Sunburst(
    df=df_sunburst,
    path_columns=['origin', 'cyl_category', 'weight_range'],
    value_column='horsepower',
    width=800,
    height=800,
    color_scheme='schemeSet3'
)

<IPython.core.display.Javascript object>

In [35]:
# Celda 4: STREAM GRAPH - Evolución de caballos de fuerza por año y origen
# CORREGIDO: usar 'model_year' en lugar de 'year'
df_stream = df.groupby(['model_year', 'origin'])['horsepower'].mean().reset_index()

StreamGraph(
    df=df_stream,
    x_column='model_year',
    y_column='horsepower',
    category_column='origin',
    width=950,
    height=500,
    color_scheme='schemeTableau10'
)

<IPython.core.display.Javascript object>

In [36]:
# Celda 5: STREAM GRAPH - Evolución de eficiencia (MPG) por año y cilindros
df_stream2 = df.copy()
df_stream2['cyl_cat'] = df_stream2['cylinders'].astype(str) + ' cyl'
df_stream2 = df_stream2.groupby(['model_year', 'cyl_cat'])['mpg'].mean().reset_index()

StreamGraph(
    df=df_stream2,
    x_column='model_year',
    y_column='mpg',
    category_column='cyl_cat',
    width=950,
    height=550,
    color_scheme='schemeCategory10'
)

<IPython.core.display.Javascript object>

In [37]:
# Celda 6: RADAR CHART - Comparación de características por origen
radar_data = df.groupby('origin')[['horsepower', 'weight', 'acceleration', 'mpg', 'displacement']].mean().reset_index()
radar_data.columns = ['name', 'Caballos', 'Peso', 'Aceleración', 'Eficiencia (MPG)', 'Cilindrada']

# Normalizar para mejor visualización en radar
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 100))
features_to_scale = ['Caballos', 'Peso', 'Aceleración', 'Eficiencia (MPG)', 'Cilindrada']
radar_data[features_to_scale] = scaler.fit_transform(radar_data[features_to_scale])

RadarChart(
    df=radar_data,
    features=features_to_scale,
    name_col='name',
    width=750,
    height=750
)

<IPython.core.display.Javascript object>

In [38]:
# Celda 7: SUNBURST alternativo - Década → Origen → Eficiencia
df_sunburst2 = df.copy()

# Crear décadas
df_sunburst2['decade'] = (df_sunburst2['model_year'] // 10 * 10).astype(str) + 's'

# Categorizar eficiencia
df_sunburst2['efficiency'] = pd.cut(
    df_sunburst2['mpg'], 
    bins=[0, 20, 30, 50], 
    labels=['Baja (<20 mpg)', 'Media (20-30)', 'Alta (>30)']
)

Sunburst(
    df=df_sunburst2,
    path_columns=['decade', 'origin', 'efficiency'],
    value_column='weight',
    width=750,
    height=750,
    color_scheme='schemePastel1'
)

<IPython.core.display.Javascript object>

In [39]:
# Celda 8: STREAM GRAPH - Evolución del peso promedio por año y origen
df_stream3 = df.groupby(['model_year', 'origin'])['weight'].mean().reset_index()

StreamGraph(
    df=df_stream3,
    x_column='model_year',
    y_column='weight',
    category_column='origin',
    width=950,
    height=550,
    color_scheme='schemeAccent'
)

<IPython.core.display.Javascript object>

In [40]:
# Celda 9: SCATTER tradicional - Caballos vs Peso (coloreado por origen)
ScatterBrush(
    df=df, 
    x="horsepower", 
    y="weight", 
    color="origin",
    width=900,
    height=600
)

<IPython.core.display.Javascript object>

In [41]:
# Celda 10: RADAR CHART - Comparación por décadas
radar_decades = df.copy()
radar_decades['decade'] = (radar_decades['model_year'] // 10 * 10).astype(str) + 's'
radar_decades_agg = radar_decades.groupby('decade')[['horsepower', 'weight', 'acceleration', 'mpg']].mean().reset_index()
radar_decades_agg.columns = ['name', 'Potencia', 'Peso', 'Aceleración', 'Eficiencia']

# Normalizar
features_radar = ['Potencia', 'Peso', 'Aceleración', 'Eficiencia']
scaler2 = MinMaxScaler(feature_range=(0, 100))
radar_decades_agg[features_radar] = scaler2.fit_transform(radar_decades_agg[features_radar])

RadarChart(
    df=radar_decades_agg,
    features=features_radar,
    name_col='name',
    width=700,
    height=700
)

<IPython.core.display.Javascript object>